# Predicting Malaria in Nigeria with Machine Learning

## Spatial Analysis

### Contributors: Brian Liggio, Jessica Sickles, Prashant Toteja, Keely Meyers

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import numpy
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import statsmodels.formula.api as smf

from gridsearch import get_results
import time
import itertools
from sklearn.linear_model import Ridge
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/prashanttoteja/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [67]:
def feature_to_array(file_n):
    #The input of the function is a raster file with dimentions (n pixels,m pixels), 
    #and the output is going to be an array of dimmensions (m*n,1). 
    #Each pixel is going to have a unique identifier (index in the array). 
    #This index follows the formula (i*n)+j where (i,j) is the position of the pixel in the image 
    #and n is the width of the image.
    im = mpimg.imread(file_n)  
    im_greyscale = rgb2gray(im)   
    imarray = numpy.array(im_greyscale)
    feature=[a for a in imarray]
    return feature

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])   #where did these values come from? - Stackoverflow

def read_pictures(filename):
    file_name=filename
    im = mpimg.imread(file_name)  
    im_greyscale = rgb2gray(im)

In [68]:
file_name='Water_Bodies.png'
water=feature_to_array(file_name) # array1

file_name='URBAN2012.png'
urban=feature_to_array(file_name) # array4

file_name='Annual_mean_temp.png'
annual_mean_temp=feature_to_array(file_name) # array5

file_name='avg_temp_february.png'
avg_temp_feb=feature_to_array(file_name) # array6

file_name='temp_annual_range_2013.png'
temp_annual_range=feature_to_array(file_name) # array7

file_name='Primary Vegetated Areas_2000.png'
vegetation=feature_to_array(file_name) # array8

file_name='Cropland_Natural Vegetation.png'
cropland=feature_to_array(file_name) # array9

file_name='Feb_precipitation.png'
feb_precip=feature_to_array(file_name) # array7

# Y variable
file_name='prec_seasonality_013.png'
case=feature_to_array(file_name) # array2

# In or outside Nigeria
file_name='Global Land Cover.png'
land=feature_to_array(file_name) # array3

file_name='Primary Vegetated Areas_2000.png'

im = mpimg.imread(file_name)  
im_greyscale = rgb2gray(im)   
imarray = numpy.array(im_greyscale)
feature=[a for a in imarray]  #iterable one level out
feature

In [69]:
#cases = list(case/np.max(case)) # Normalize
inorout = list(1-(land/np.max(land))) # get binary value indicating whether pixel is inside Singapore or not
inorout[10]

0.0

In [71]:
Data = pd.DataFrame([water,urban,annual_mean_temp,avg_temp_feb,temp_annual_range,vegetation,cropland,feb_precip,case]).T
Data

#Data.shape

#Data

,0,1,2,3,4,5,6,7,8
0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [58]:
Data_filt = Data[np.array(inorout)==1.0] # filter dataset to only pixels within Nigeria
#Data_filt.columns = ['BusStop','Street','Trash',
                     #'Mosquito','Lot','Water','Population','Park','Case']
#Data_filt['Case'] = np.where(Data_filt['Case']==0,1,0) # Transform Case label into 1 (=case) or 0 (=no case)

ValueError: Item wrong length 1250 instead of 2307.

In [ ]:
Data_filt.head()

In [ ]:
print Data_filt.groupby('Case').size()
print (Data_filt.groupby('Case').size()/len(Data_filt))*100